In [1]:
import os
import pandas as pd
import re
import glob
from Bio import SeqIO

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/virome_ecology_core_prkaryotes/results/04_prophage_rumen_mags")

In [3]:
with open("prophage_rumen_mags_category3.txt", "r") as file:
    lines = file.readlines()
    category3 = [f.split("\n")[0] for f in lines]

with open("prophage_rumen_mags_category3.fasta", 'w') as outfile:
    records = SeqIO.parse("prophage_rumen_mags_vs2_vibrant_intersection.fa", "fasta")
    for record in records:
        if record.name in category3:
            SeqIO.write(record, outfile, "fasta")

In [4]:
with open("prophage_rumen_mags_category2.txt", "r") as file:
    lines = file.readlines()
    category2 = [f.split("\n")[0] for f in lines]

with open("prophage_rumen_mags_category2.fasta", 'w') as outfile:
    records = SeqIO.parse("prophage_rumen_mags_vs2_vibrant_intersection.fa", "fasta")
    for record in records:
        if record.name in category2:
            SeqIO.write(record, outfile, "fasta")

In [5]:
rumen_mags = glob.glob("../../../databases/rumen_mags_high_quality/dereplicated_genomes_with_taxid/*.fasta")

In [6]:
genome_seqname = {}
for f in rumen_mags:
    seqname = []
    genome = f.split("/")[-1].split(".fasta")[0]
    records = SeqIO.parse(f, "fasta")
    for record in records:
        seqname.append(record.name)
    genome_seqname[genome] = seqname
        

In [7]:
host_taxa = pd.read_csv("../../../databases/rumen_mags_high_quality/mags_taxa.tsv", sep='\t')[["user_genome", "Phylum", "Class", "Order", "Family", "Genera", "Species"]]

In [8]:
def return_key(x):
    for f in range(len(list(genome_seqname.values()))):
        if x.split("||")[0] in list(genome_seqname.values())[f]:
            return list(genome_seqname.keys())[f]
            break
        
            

In [9]:
prophage_genome = {}
for f in category3:
    prophage_genome[f] = return_key(f)

In [10]:
prophage_host_df = pd.DataFrame.from_dict(prophage_genome, orient = "index").reset_index()
prophage_host_df.rename(columns = {"index":"prophage", 0:"host_genome"}, inplace = True)

In [11]:
host_taxa = pd.merge(host_taxa, prophage_host_df, left_on = "user_genome", right_on = "host_genome", how = "left")

In [12]:
host_taxa[["Phylum", "Class", "Order", "Family", "Genera", "Species"]] = host_taxa[["Phylum", "Class", "Order", "Family", "Genera", "Species"]].applymap(lambda x: x.split("__")[1])

In [13]:
host_taxa.drop("host_genome", axis = 1, inplace = True)

In [14]:
host_taxa.to_csv("/fs/ess/PAS0439/MING/virome_ecology/results/rumen_mags_prophage/rumen_mags_prophage_summary.csv", index = None)